In [21]:
import torch
temb_dim = 4
# time_steps has to have dimension of (batch_size, )
time_steps = torch.tensor([1, 2, 3, 4, 5], dtype = torch.float32)

factor = 10000 ** ((torch.arange(
        start = 0 , end = temb_dim // 2, dtype = torch.float32) / (temb_dim // 2) )
        )

t_emb = time_steps[:, None].repeat(1, temb_dim // 2) / factor
t_emb = torch.cat([torch.sin(t_emb), torch.cos(t_emb)], dim=-1)

In [22]:
t_emb

tensor([[ 0.8415,  0.0100,  0.5403,  0.9999],
        [ 0.9093,  0.0200, -0.4161,  0.9998],
        [ 0.1411,  0.0300, -0.9900,  0.9996],
        [-0.7568,  0.0400, -0.6536,  0.9992],
        [-0.9589,  0.0500,  0.2837,  0.9988]])

In [19]:
t_emb = time_steps[:, None].repeat(1, temb_dim // 2) / factor

In [20]:
t_emb

tensor([[1.0000, 0.0100],
        [2.0000, 0.0200],
        [3.0000, 0.0300],
        [4.0000, 0.0400],
        [5.0000, 0.0500]])

In [13]:
torch.arange(start = 0 , end = temb_dim // 2, dtype = torch.float32) / (temb_dim // 2) 

tensor([0.0000, 0.5000])